In [ ]:
import prehook

prehook.execute_prehook()

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
# hook.create_etl_checkpoint(db_session)

In [ ]:
hook.return_lookup_items_as_dict(lookups.IncrementalField)

In [ ]:
import hook
import prehook
import database_handler
import lookups
import datetime

db_session = database_handler.create_connection()
etl_date = datetime.datetime(1900,1,1)
#SourceName is schema name
source_name = lookups.SourceName.DVD_RENTAL.value
#This will get you a list of tables inside this schema that are mentioned in SQLTablesToReplicate
tables = prehook.return_tables_by_schema(source_name)
#This will return 
incremental_date_dict = hook.return_lookup_items_as_dict(lookups.IncrementalField)
# for table in tables:
staging_query = f"""
        SELECT * FROM {source_name}.{tables[0]} WHERE {incremental_date_dict.get(tables[0])} >= '{etl_date}'
""" 
staging_df = database_handler.return_data_as_df(db_session= db_session, input_type= lookups.InputTypes.SQL, file_executor= staging_query)
dst_table = f"stg_{source_name}_{tables[0]}"
insert_stmt = database_handler.return_insert_into_sql_statement_from_df(staging_df, 'dw_reporting', dst_table)
for insert in insert_stmt:
        database_handler.execute_query(db_session=db_session, query= insert)

In [1]:
import misc_handler


In [2]:
df = misc_handler.return_match_df_from_web(94912,94913)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6BF857892+54818]
	(No symbol) [0x00007FF6BF7C6AC2]
	(No symbol) [0x00007FF6BF67DA3B]
	(No symbol) [0x00007FF6BF6BE4FC]
	(No symbol) [0x00007FF6BF6BE67C]
	(No symbol) [0x00007FF6BF6F9627]
	(No symbol) [0x00007FF6BF6DEAEF]
	(No symbol) [0x00007FF6BF6F75A2]
	(No symbol) [0x00007FF6BF6DE883]
	(No symbol) [0x00007FF6BF6B3691]
	(No symbol) [0x00007FF6BF6B48D4]
	GetHandleVerifier [0x00007FF6BFBBB992+3610402]
	GetHandleVerifier [0x00007FF6BFC11860+3962352]
	GetHandleVerifier [0x00007FF6BFC09D4F+3930847]
	GetHandleVerifier [0x00007FF6BF8F3646+693206]
	(No symbol) [0x00007FF6BF7D1628]
	(No symbol) [0x00007FF6BF7CD934]
	(No symbol) [0x00007FF6BF7CDA62]
	(No symbol) [0x00007FF6BF7BE113]
	BaseThreadInitThunk [0x00007FFE82837344+20]
	RtlUserThreadStart [0x00007FFE839A26B1+33]


In [ ]:
df['Date'] = df['Date'].astype('datetime64[ns]')

In [ ]:
df.columns=df.columns.str.replace("_%","")

In [ ]:
df

In [ ]:
from prehook import first_time_web_scraping
first_time_web_scraping()